In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import torch
torch.__version__


'1.12.1+cu113'

In [2]:
torch.cuda.is_available()

True

In [3]:
from transformers import BertForTokenClassification, BertTokenizer
import json
import numpy as np
from keras.preprocessing.sequence import pad_sequences

c:\Users\Taimoor Khan\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:


with open('/content/drive/My Drive/Bert_model/tag2idx.json', 'r', encoding='utf-8') as f:
    tag2idx = json.load(f)
idx2tag = {i: tag for tag, i in tag2idx.items()}
# Load the model
model_path = '/content/drive/My Drive/Bert_model/Ubert_model.h5'
model = BertForTokenClassification.from_pretrained(
    'bert-base-multilingual-cased',
    num_labels=len(tag2idx),  # You need to define or load tag2idx as done earlier
    state_dict=torch.load(model_path)
)
model.eval()  # Set the model to evaluation mode

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('/content/drive/My Drive/Bert_model/tokenizer')

# Load tag2idx


# Load metadata
with open('/content/drive/My Drive/Bert_model/metadata.json', 'r', encoding='utf-8') as f:
    metadata = json.load(f)
MAX_LEN = metadata['MAX_LEN']


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [2]:
def predict_tags(sentence, model, tokenizer, max_len, idx2tag):
    # Tokenize the sentence
    inputs = tokenizer.encode_plus(sentence, add_special_tokens=True, max_length=max_len,
                                   padding='max_length', return_tensors='pt', truncation=True)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    # Move tensors to the same device as the model
    input_ids = input_ids.to(model.device)
    attention_mask = attention_mask.to(model.device)

    # Predict
    model.eval()
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits

    # Convert logits to list of tags
    predictions = torch.argmax(logits, dim=2)
    predictions = predictions[0].cpu().numpy()  # Take the first sentence (batch)

    # Token to word mapping
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0].cpu().numpy())
    new_tokens, new_labels = [], []
    for token, label_idx in zip(tokens, predictions):
        if token.startswith("##"):
            new_tokens[-1] = new_tokens[-1] + token[2:]
        else:
            new_labels.append(idx2tag[label_idx])
            new_tokens.append(token)

    # Filter out special tokens
    token_tag_pairs = [(token, tag) for token, tag in zip(new_tokens, new_labels) if token not in [tokenizer.cls_token, tokenizer.sep_token, tokenizer.pad_token]]

    return token_tag_pairs

# Example usage
test_sentence = "پروفیسر احمد خان، نینو ٹیکنالوجیز کے ڈائریکٹر، سارہ علی، ریحان احمد اور فاطمہ زہرا کے ساتھ انٹرنیشنل ٹیک انوویشن سمٹ 2023 میں شریک ہوں گے، جو کہ 15 اکتوبر کو شام 5:00 بجے کراچی میں منعقد ہوگی۔ اس سمٹ میں وہ متعدد دیگر ممتاز ایگزیکٹوز کے ساتھ موجود ہوں گے"
predicted_token_tags = predict_tags(test_sentence, model, tokenizer, MAX_LEN, idx2tag)
for token, tag in predicted_token_tags:
    print(f"('{token}', '{tag}')")




NameError: name 'model' is not defined

In [9]:
def merge_same_tags(token_tag_pairs):
    merged_pairs = []
    current_word, current_tag = None, None

    for word, tag in token_tag_pairs:
        if tag == current_tag:
            # Append word to the current word
            current_word += ' ' + word
        else:
            # Add the previous word (if any) to the result
            if current_word:
                merged_pairs.append((current_word, current_tag))
            current_word, current_tag = word, tag

    # Don't forget to add the last word
    if current_word:
        merged_pairs.append((current_word, current_tag))

    return merged_pairs


merged_token_tags = merge_same_tags(predicted_token_tags)
for word, tag in merged_token_tags:
    print(f"('{word}', '{tag}')")

('پروفیسر', 'Designation')
('احمد خان', 'Person')
('،', 'Other')
('نینو ٹیکنالوجیز', 'Organization')
('کے', 'Other')
('ڈائریکٹر', 'Designation')
('،', 'Other')
('سارہ علی', 'Person')
('،', 'Other')
('ریحان احمد', 'Person')
('اور', 'Other')
('فاطمہ زہرا', 'Person')
('کے ساتھ انٹرنیشنل ٹیک انوویشن سمٹ', 'Other')
('2023', 'Date')
('میں شریک ہوں گے ، جو کہ', 'Other')
('15 اکتوبر', 'Date')
('کو', 'Other')
('شام 5 : 00 بجے', 'Time')
('کراچی', 'Location')
('میں منعقد ہوگی ۔ اس سمٹ میں وہ متعدد دیگر ممتاز ایگزیکٹوز کے ساتھ موجود ہوں گے', 'Other')
